In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
from umap import UMAP
from trimap import TRIMAP
from pacmap import PaCMAP

M:\Anaconda\envs\tf\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
M:\Anaconda\envs\tf\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
M:\Anaconda\envs\tf\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argument was

generowanie fig - to będzie do zmiany

In [2]:

## kod pożyczony
# (można sprawdzić, czy napewno dobry w wielu wymiarach)
# torus
def generate_n_dimensional_torus(n_dimensions, n_points, R=2, r=1):
    """
    Creates an n-dimensional torus.

    Parameters:
    n_dimensions : int
        The number of dimensions for the torus. Must be greater than or equal to 2.
    n_points : int
        The number of points.
    R : float
        The distance from the center of the tube to the center of the torus.
    r : float
        The radius of the tube (distance from the center of the tube to the torus surface).

    Returns:
    numpy.ndarray
        The n-dimensional points of the torus.
    """

    assert n_dimensions >= 2, "Number of dimensions must be greater than or equal to 2."

    # generate n_points random angles for each dimension
    angles = np.random.uniform(0, 2*np.pi, (n_points, n_dimensions))

    # calculate the n-dimensional points on the torus
    coordinates = []
    for i in range(n_dimensions):
        if i == 0:
            coordinate = (R + r * np.cos(angles[:, 1])) * np.cos(angles[:, 0])
        elif i == 1:
            coordinate = (R + r * np.cos(angles[:, 1])) * np.sin(angles[:, 0])
        else:
            coordinate = r * np.sin(angles[:, i])
        coordinates.append(coordinate)

    return np.array(coordinates).T

#https://blogs.sas.com/content/iml/2016/04/06/generate-points-uniformly-in-ball.html
def ball_points_generator(dim, N, radius):
    # Generowanie macierzy zmiennych losowych o rozkładzie normalnym standardowym
    xi = np.random.normal(size=(dim, N))

    # obliczanie normy
    norm = np.linalg.norm(xi, axis=0)

    S = xi / norm
    U = np.random.uniform(low=0.0, high=1.0, size=N)
    S = S * np.power(U, 1/dim)
    points = S * radius

    # Transponowanie macierzy punktów
    # points = scaled_xi.T

    return points.T

def sphere_points_generator(dim, N, radius):
    # Generowanie macierzy zmiennych losowych o rozkładzie normalnym standardowym
    xi = np.random.normal(size=(dim, N))

    # Obliczanie sumy kwadratów zmiennych losowych dla każdego punktu
    sum_of_squares = np.sum(xi**2, axis=0)

    # Obliczanie wartości λ na podstawie żądanej wartości
    lambda_squared = radius**2

    # Skalowanie zmiennych losowych dla każdego punktu, aby uzyskać żądaną wartość λ
    scaled_xi = xi * np.sqrt(lambda_squared / sum_of_squares)

    # Transponowanie macierzy punktów
    # unit_vectors = scaled_xi.T

    # return unit_vecto
    return scaled_xi.T

redukcja wymiarowości

In [3]:
def TSNE_reduce(data):
    tsne = TSNE()
    return tsne.fit_transform(data), "TSNE"

def Umap_reduce(data):
    umap = UMAP()
    return umap.fit_transform(data), "Umap"

def Trimap_reduce(data):
    trimap = TRIMAP()
    return trimap.fit_transform(data), "Trimap"

def Pacmap_reduce(data):
    pacmap = PaCMAP()
    return pacmap.fit_transform(data), "Pacmap"

# def IVHD_reduce(data):
#     ...



In [5]:
## losowe generowanie

#kulka i 2 sfery
def ball_two_sphere(dim,points):
    sphere_points = sphere_points_generator(dim, points, 5)
    sphere_points2 = sphere_points_generator(dim, points, 7)
    ball_points = ball_points_generator(dim, points, 2)
    y1 = np.zeros(points)
    y2 = np.ones(points)
    y3 = np.full(points, 2)
    labels = np.concatenate((y1, y2, y3))
    return np.concatenate((sphere_points, sphere_points2, ball_points)), 3, labels

#one sphere one ball
def ball_sphere(dim,points):
        sphere_points = sphere_points_generator(dim, points, 5)
        ball_points = ball_points_generator(dim, points, 2)
        y1 = np.zeros(points)
        y2 = np.ones(points)
        labels = np.concatenate((y1, y2))
        return  np.concatenate((sphere_points, ball_points)), 2, labels


def one_sphere_one_torus_inside_dataset( dim, points_number, sphere_r=4, torus_R=2, torus_r=1):

     sphere_points = sphere_points_generator(dim, points_number, sphere_r)
     torus_points = generate_n_dimensional_torus(dim, points_number, torus_R, torus_r)
     y1 = np.zeros(points_number)
     y2 = np.ones(points_number)

     labels = np.concatenate((y1, y2))
     return np.concatenate((sphere_points, torus_points)), 4, labels


In [7]:
import os
import matplotlib.pyplot as plt

from local_score import LocalMetric
reduction_dict = { "TSNE": TSNE_reduce,"Umap":Umap_reduce,"Trimap":Trimap_reduce,"Pacmap":Pacmap_reduce}
figure_dict = {"ball_sphere" : ball_sphere, "ball_two_sphere": ball_two_sphere, "one_sphere_one_torus_inside_dataset":one_sphere_one_torus_inside_dataset}


dimensions = [3, 5, 10, 20]
emem = [1000, 2000, 5000, 20000]

local_metrics_list = []

for _ in enumerate(dimensions):
    local_metrics_list.append(LocalMetric())




def show_plot(reduction_method, data):

    colors = ['b', 'g', 'r']  # Define colors for each set

    fig, axes = plt.subplots(len(dimensions), len(emem))
    plt_name = ""
    for j, e in enumerate(emem):
        for i, d in enumerate(dimensions):


            dataset, object_count, labels = data(d,e)
            reduced, red_name = reduction_method(dataset) #to
            if e == max(emem):
                local_metrics_list[i].calculate_knn_gain_and_dr_quality(
                X_lds=reduced,
                X_hds=dataset,
                labels=labels,
                method_name=f"{red_name}")
            #generate_metrics(reduced)
            # print(type(dataset))
            # print(type(reduced))
            # Plot in the corresponding subplot
            axes[i, j].set_title(f"dim: {d}, elem: {e}")

            fig_name = ''

            # #if object_count == 2:
            if object_count == 2:
            #     # Assign colors based on the sets
                axes[i, j].scatter(reduced[:e, 0], reduced[:e, 1], c=colors[0], alpha=0.5, label='Sphere Points', s=1)
                axes[i, j].scatter(reduced[e:, 0], reduced[e:, 1], c=colors[1], alpha=0.5, label='Ball Points', s=1)
                fig_name = "ball_sphere"
            elif object_count == 3:
                axes[i, j].scatter(reduced[:e, 0], reduced[:e, 1], c=colors[1], alpha=0.5, label='Sphere Points', s=1)
                axes[i, j].scatter(reduced[e:2*e, 0], reduced[e:2*e, 1], c=colors[0], alpha=0.5, label='Sphere Points', s=1)
                axes[i, j].scatter(reduced[2*e:, 0], reduced[2*e:, 1], c=colors[2], alpha=0.5, label='Ball Points', s=1)
                fig_name ="ball_two_sphere"
            else:
                axes[i, j].scatter(reduced[:e, 0], reduced[:e, 1], c=colors[0], alpha=0.5, label='Sphere Points', s=1)
                axes[i, j].scatter(reduced[e:, 0], reduced[e:, 1], c=colors[1], alpha=0.5, label='torus_points', s=1)
                fig_name = "one_sphere_one_torus_inside_dataset"

            # Save the plot as a distinct figure
            folder_path = f"datasets/{fig_name}/{red_name}"
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

            # Zapis plików
            np.savetxt(f"{folder_path}/EMB_dim_{d}_points_{e}.csv", reduced, delimiter=",")
            np.savetxt(f"{folder_path}/DATA_dim_{d}_points_{e}.csv", dataset, delimiter=",")

    plt.tight_layout()
    handles, labels = axes[0, 0].get_legend_handles_labels()
    fig.legend(handles, labels)
    fig.set_size_inches(18, 12)
    plt_name = f'plots_final/{red_name}_{fig_name}.png'
    plt.savefig(plt_name)
    plt.close(fig)  # Close the figure to release memory

    print("Plots saved successfully!")

In [8]:
#show_plot(Trimap_reduce,ball_two_sphere)

In [ ]:


for figure_name, generate_figure in figure_dict.items():
    print(figure_name)
    for reduction_name, reduction_m in reduction_dict.items():

        show_plot(reduction_m, generate_figure)  # Show plot function is not defined
        print(reduction_name)

    for i, e in enumerate(dimensions):
        print(f"emuerta{i} dimm {e}")
        #local_metrics_list[i].visualize(dimensions[i], figure_name)
        local_metrics_list[i].visualize(f"{figure_name} dimm {e}")

    local_metrics_list.clear()
    for _ in enumerate(dimensions):
        local_metrics_list.append(LocalMetric())


ball_sphere
Calculating d_hd
TSNE
Calculating d_hd
TSNE
Calculating d_hd
TSNE
Calculating d_hd
TSNE
Plots saved successfully!
TSNE
Calculating d_hd
Umap
Calculating d_hd
Umap
Calculating d_hd
Umap
Calculating d_hd
Umap
Plots saved successfully!
Umap
Calculating d_hd
Trimap
Calculating d_hd
Trimap
Calculating d_hd
Trimap
Calculating d_hd
Trimap
Plots saved successfully!
Trimap
Calculating d_hd
Pacmap
Calculating d_hd
Pacmap
Calculating d_hd
Pacmap
Calculating d_hd
Pacmap
Plots saved successfully!
Pacmap
emuerta0 dimm 3
Finished.
emuerta1 dimm 5
Finished.
emuerta2 dimm 10
Finished.
emuerta3 dimm 20
Finished.
ball_two_sphere
Calculating d_hd
TSNE
